In [52]:
import sys
from pathlib import Path

p = Path.cwd().resolve()
repo_root = next((parent for parent in [p] + list(p.parents) if (parent / ".git").exists()), None)
if repo_root is None:
    raise RuntimeError("Repo root not found. Open the repo folder in VS Code.")

sys.path.insert(0, str(repo_root))
print("Repo root:", repo_root)

Repo root: /Users/samiullah/Group_Project_Y3


Imports
/usr/local/bin/python3 -m pip install [package]

In [53]:
import pandas as pd

In [54]:
county_codes = pd.read_csv(f"{str(repo_root)}/cleaned/00_codes/county_codes.csv")
county_codes

,county_code,county_kts,county_name
0,201,10030210101000,Powiat bolesławiecki
1,202,10030210302000,Powiat dzierżoniowski
2,203,10030210203000,Powiat głogowski
3,204,10030210204000,Powiat górowski
4,205,10030210105000,Powiat jaworski
...,...,...,...
375,3217,10023216417000,Powiat wałecki
376,3218,10023216418000,Powiat łobeski
377,3261,10023216361000,Powiat m. Koszalin
378,3262,10023216562000,Powiat m. Szczecin


In [55]:
citizenship = pd.read_excel(f"{str(repo_root)}/raw/00_prewar_ukrainians/NARO_4298_XREL_20260206125502.xlsx", sheet_name="DATA")

In [56]:
birthplace = pd.read_excel(f"{str(repo_root)}/raw/00_prewar_ukrainians/NARO_4299_XREL_20260206125233.xlsx", sheet_name="DATA")

In [57]:
def clean_table(df, name_col):
    df_ukr = df[(df['Name'].str.lower().str.contains('powiat')) & (df['Countries'] == 'Ukraine') & (df['Sex']=='total')]
    df_ukr['county_kts'] = ("100" + df_ukr['Code'].astype(str)).astype(int) # convert to kts

    df_ukr['total_ukr'] = df[(df['Name'] == 'POLAND') & (df['Sex'] == 'total') & (df['Countries'] == 'Ukraine')]['Value'].iloc[0]

    df_ukr = df_ukr.merge(county_codes, on='county_kts', how='right')
    df_ukr = df_ukr[['county_code', 'county_name', 'Value', 'total_ukr']].rename(columns={'Value': name_col})

    df_ukr[f'{name_col}_share'] = df_ukr[name_col] / df_ukr['total_ukr']

    return df_ukr

In [58]:
citizenship_ukr = clean_table(citizenship, 'ukr_citizens')
citizenship_ukr.to_csv(f"{str(repo_root)}/cleaned/00_prewar_ukrainian_shares/citizenship_ukr.csv", index=False)

In [59]:
birthplace_ukr = clean_table(birthplace, 'ukr_birthplace')
birthplace_ukr.to_csv(f"{str(repo_root)}/cleaned/00_prewar_ukrainian_shares/birthplace_ukr.csv", index=False)

In [ ]:
print('ch')